# Pratique de XSL : de XML vers CSV

## Présentation

**CSV :** *Comma-Separated Values*

Il s’agit d’un type de fichier à plat (*flat file*) autrement nommé base de données orientée texte, très facile à manipuler, notamment via un terminal.

```csv
4,PUC,Paris,22,13,09,38
8,TVB,Tours,22,15,07,45
7,NVB,Nice,22,10,12,31
```

Les fichiers CSV sont constitués de lignes et de colonnes où chaque ligne représente un enregistrement et où les colonnes sont matérialisées par un caractère séparateur (la virugle par défaut).

```txt
<!-- Représentation tabulaire -->
|---|-----|-------|----|----|----|----|
| 4 | PUC | Paris | 22 | 13 | 09 | 38 |
|---|-----|-------|----|----|----|----|
| 8 | TVB | Tours | 22 | 15 | 07 | 45 |
|---|-----|-------|----|----|----|----|
| 7 | NVB | Nice  | 22 | 10 | 12 | 31 |
|---|-----|-------|----|----|----|----|
```

Les sytèmes de gestion de bases de données relationnelles exportent et importent souvent leurs données grâce aux fichiers à plats.

## Outils en ligne de commande

Il existe une multitude d’outils en ligne de commande pour interroger directement les fichiers à plat :

```shell
# number the lines
$ cat -n fichier.csv
```

```shell
# randomly swap lines
$ cat -n fichier.csv | sort --random-sort
```

```shell
# extract the first three columns
$ cut -f 1,2,3 -d, fichier.csv
```

## La structure d’une feuille de transformation en CSV

```xml
<?xml version="1.0" encoding="UTF-8"?>
<!-- root -->
<xsl:stylesheet version="3.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

    <!-- csv is a text document -->
    <xsl:output method="text" encoding="utf-8"/>
  
    <!-- main template -->
    <xsl:template match="/">
        <!-- call -->
        <xsl:apply-templates select="{XPath}"/>
    </xsl:template>

</xsl:stylesheet>
```

## Exemple

### Fichier XML

```xml
<?xml version="1.0" encoding="UTF-8"?>
<equipes>
    <equipe num="4" nom="PUC" ville="Paris">
        <matchs>22</matchs>
        <victoires>13</victoires>
        <defaites>9</defaites>
        <points>38</points>
    </equipe>
    <equipe num="8" nom="TVB" ville="Tours">
        <matchs>22</matchs>
        <victoires>15</victoires>
        <defaites>7</defaites>
        <points>45</points>
    </equipe>
    <equipe num="7" nom="NVB" ville="Nice">
        <matchs>22</matchs>
        <victoires>10</victoires>
        <defaites>12</defaites>
        <points>31</points>
    </equipe>
</equipes>
```

### Résultat attendu

```csv
4,PUC,Paris,22,13,09,38
8,TVB,Tours,22,15,07,45
7,NVB,Nice,22,10,12,31
```

### Transformations

**Objectifs :**
- chaque élement `equipe` doit figurer sur une sur ligne (il s’agit de l’unité de base, l’enregistrement) ;
- les attributs et les éléments fils sont considérés comme les colonnes.

#### 1e étape : paramétrer les éléments textuels

```xml
<xsl:param name="sep" select="','"/>
<xsl:param name="break" select="'&amp;#10;'"/>
```

#### 2e étape : appel à candidature

```xml
<xsl:template match="/">

    <!-- all attributes AND elements which are children of "Equipe" -->
    <xsl:apply-templates select="//equipe/@* | //equipe/*"/>

</xsl:template>
```

#### 3e étape : template candidat

```xml
<!-- candidate for attributes and elements -->
<xsl:template match="@* | *">

    <!-- copy of textual content -->
    <xsl:value-of select="."/>

    <xsl:choose>

        <!-- while node name differs from "points" -->
        <xsl:when test="name() != 'points'">
            <!-- delimiter -->
            <xsl:value-of select="$sep"/>
        </xsl:when>

        <xsl:otherwise>
            <!-- otherwise: break line -->
            <xsl:value-of select="$break"/>
        </xsl:otherwise>

    </xsl:choose>

</xsl:template>
```